## Fine-Tuning the GPTJ-6B model using transfer learning

## Overview
This notebook will walk you through how to fine-tune a pre-trained large language model with domain specific knowledge. 

The domain specific dataset that we will be using to fine-tune this model will be United Kingdom (U.K.) Supreme Court case documents. We will tune the model on roughly 693 legal documents. 

## Dataset info
* <strong>Page count:</strong> ~17,718
* <strong>Word count:</strong> 10,015,333
* <strong>Characters (no spaces):</strong> 49,897,639

The entire dataset is publically available and can be download [here](https://zenodo.org/record/7152317#.ZCSfaoTMI2y)

## Considerations when fine-tuning the model
The notebook has been configured to allow you to only use a subset of the entire dataset to fine-tune the model if you would like. There is a variable named _**doc_count**_ in the _**Data Prep**_ section. You can set this number to whatever you would like and it will only fine-tune the model based on the number of documents you set this variable to. The smaller this value the faster the model will fine-tune.
    
## Training/Tuning Time estimates

Here are the estimated training times based on total number of case documents in the training dataset.

#### All training was ran on 1 - *ml.p3dn.24xlarge* instance

#### <strong>Training dataset document count </strong> 250
Training time: 1 hour 41 minutes

#### <strong>Training document count</strong> 500
Training time: 2 hours 57 minutes

#### <strong>Training document count</strong> 693
Training time: 4 hours

## GPTJ-6B base model

Steps you will go through in the notebook to test the base model

1. Clone this repo in a SageMaker Studio Jupyter notebook
2. Install needed notebook libraries
3. Configure the notebook to use SageMaker
4. Retrieve base model container
5. Deploy the model inference endpoint
6. Call inference endpoint to retrieve results from the LLM

## Fine-tuned model

Steps you will go through in the notebook to test the fine-tuned model

1. Download dataset
2. Prep the dataset and upload it to S3
3. Retrieve the base model container
4. Set hyperparameters for fine-tuning
5. Start training/tuning job
6. Deploy inference endpoint for the fine-tuned model
7. Call inference endpoint for the fine-tuned model
8. Parse endpoint results

### Final Step
* Be sure you delete all models and endpoints to avoid incurring unneeded spend.
    
### Disclaimer
This notebook demos how you can fine-tune an LLM using transfer learning. Even though this notebook is fine-tuned using actual (U.K.) Supreme Court case documents you should not use this notebook for legal advise.
    
    

## Install Pre Reqs

In [ ]:
!pip install --upgrade sagemaker --quiet

## SageMaker SDK configurations

In [ ]:
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()
account_id = sess.account_id()

print(f"Role SageMaker will use - {aws_role}")
print(f"AWS Account ID: {account_id}")

## Deploying interence endpoint for the GPTJ-6 base model

In this section we are deploying the HuggingFace GPTJ-6B base model in order to compare the inference results with the fine-tuned model we will tune later.

The fine-tuned model will be trained on UK Supreme Court case documents.

In [ ]:
# Name of model being used
model_id, model_version = "huggingface-textgeneration1-gpt-j-6b", "*"

In [ ]:
from sagemaker import image_uris, model_uris, script_uris
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base

endpoint_name = name_from_base(f"base-model-gptj-6B-{model_id}")

inference_instance_type = "ml.g5.12xlarge"

# Retrieve the inference docker container uri.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)

print(f"Container location: {deploy_image_uri}")

# Retrieve the model uri.
model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="inference"
)

print(f"Model location: {model_uri}")

# Create the SageMaker model instance. Note that we need to pass Predictor class when we deploy model through Model class,
# for being able to run inference through the sagemaker API.
model = Model(
    image_uri=deploy_image_uri,
    model_data=model_uri,
    role=aws_role,
    predictor_cls=Predictor,
    name=endpoint_name,
)

# deploy the base model
base_model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    endpoint_name=endpoint_name,
)

print(f"Endpoint name: {endpoint_name}" )

## Inference Helper functions
Creates two helper functions that will be used when we call the inference endpoint

In [ ]:
import json
import boto3

def query_endpoint_with_payload(encoded_json, endpoint_name):
    client = boto3.client("runtime.sagemaker")
    
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/json", Body=encoded_json
    )
    
    return response


def parse_response_texts(query_response):
    generated_text = []
    model_predictions = json.loads(query_response["Body"].read())
    return model_predictions[0]

## Call GPTJ-6B without fine-tuning the model
In this section we make a call to the SageMaker inference point that host the base model and have the results returned back from the endpoint.

In [ ]:
parameters = {
    "max_length": 500,
    "num_return_sequences": 1,
    "top_k": 250,
    "top_p": 0.8,
    "do_sample": True,
    "temperature": 1,
}

res_gpt_before_finetune = []

for quota_text in [
    "Tell me about the Matrimonial and Family Proceedings Act 1984",
    "tell me about the Ellenborough Park [1956] case",
    # "what was the case about that involved The Palmers Wood Oil Field",
    # "what was the case about that involved Mohammed Jabar Ahmed, Mohammed Azmir Khan and Michael Marteen",
    # "Tel me about Thirteenth Protocol to the European Convention on Human Rights (2004)",
]:
    
    payload = {"text_inputs": f"{quota_text}:", **parameters}

    query_response = query_endpoint_with_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    
    generated_texts = parse_response_texts(query_response)[0]["generated_text"]
    res_gpt_before_finetune.append(generated_texts)
    
    print(generated_texts)
    print("\n----------------------------")

### Base model results
The output above is what the base model will return to use before fine-tuning the model. It will only return with data that it knows about when the model was pre-trained. The goal is to make the model give us better results after it has more context based on case law that it will be fine-tuned with.

## Clean-up

Delete the SageMaker endpoint and the attached resources once you no longer endpoint them. The inteference endpoints incur cost if you leave them running.

In [ ]:
base_model_predictor.delete_model()
base_model_predictor.delete_endpoint()

# Fine-Tuning the GPTJ-6 base model via transfer learning

## Data Prep

Download the dataset. This may take several minutes. The zipped dataset is 93 MB.

In [ ]:
!wget https://zenodo.org/record/7152317/files/dataset.zip

In [ ]:
# unzipping compressed datasets
# this may take several minutes since we are decompressing all the case files in the dataset

print("Unzipping file. Wait for the dataset files to unzip. This may take several minutes ...")

!unzip -q dataset.zip

print("Finished unzipping file")

## Configure file path and storage locations

In [3]:
import os

# Replace 'path/to/your/directory' with the actual path to your directory containing the text files
training_path = 'dataset/UK-Abs/train-data/judgement'
validation_path = 'dataset/UK-Abs/train-data/summary'

local_training_file = 'dataset/train.txt'
s3_training_file = "train.txt"
s3_validation_file = "validation.txt"

# Replace 'new_file.txt' with the name of the new file where you want to combine the contents
training_file_path = 'dataset/train.txt'
validation_file_path = 'dataset/validation.txt'

model = "gptj-6b"
bucket_name = f'sagemaker-{account_id}-{aws_region}' # change this to your bucket name and be sure it exist in S3
training_folder = f'{model}/train' # the training folder in your bucket
validation_folder = f'{model}/validation' # the training folder in your bucket

s3_training_location = f"s3://{bucket_name}/{training_folder}/"
s3_validation_location = f"s3://{bucket_name}/{validation_folder}/"
s3_training_output_path = f"s3://{bucket_name}/{model}/output"

training_dataset = f"{s3_training_location}{s3_training_file}"
validation_dataset = f"{s3_validation_location}{s3_validation_file}"

s3_output_location = f"s3://{bucket_name}/{model}/output"

## Creating Training dataset

In [4]:
# doc_count is the number of documents to include the fine-tuning dataset
# The higher the doc_count the larger the training dataset will be
# The max document count is 693
doc_count = 10

def create_dataset(new_dataset_file, training_path, docs_in_dataset):
    doc_in_dataset = 0
    
    with open(new_dataset_file, 'w') as new_file:
        file_list = os.listdir(training_path)

        for filename in file_list:
            if doc_in_dataset < doc_count:
                doc_in_dataset+=1
                # Create the full file path by joining the directory path with the filename
                file_path = os.path.join(training_path, filename)

                # Check if the file is a regular file (not a directory)
                if os.path.isfile(file_path):
                    # Open the file in read mode
                    with open(file_path, 'r') as file:
                        text_content = file.read()

                    # Write the content of each file to the new file
                    new_file.write(text_content)
                    new_file.write("\n-----------------------------------------------------------------\n")
                    
    print(f"Local {new_file.name} dataset created")

# creats training dataset
create_dataset(training_file_path, training_path, doc_count)

# creats validation dataset
create_dataset(validation_file_path, validation_path, doc_count)

Local dataset/train.txt dataset created
Local dataset/validation.txt dataset created


## Upload training data to S3
In this section we upload the dataset that was created in the previous step

In [ ]:
# uploads training data to S3 so that model can be fine-tune using the dataset
sagemaker_session.upload_data(local_training_file,
                              bucket=bucket_name, 
                              key_prefix=training_folder)

sagemaker_session.upload_data(validation_file_path,
                              bucket=bucket_name, 
                              key_prefix=validation_folder)

print(f"S3 Training location: {s3_training_location}")
print(f"S3 Validation location: {s3_validation_location}")
print(f"S3 Training dataset file: {training_dataset}")
print(f"S3 Validation dataset file: {validation_dataset}")
print(f"Model training output location: {s3_output_location}")

print("Training data uploaded to S3")

## Setup Model to be tuned

When selecting your instance type below ensure you have the minimal available to run based on your account quota. For some GPU based instances you may need to request an increase in the total number you can run in your account. This is true for spot instance type also which have a separate quota. 

You can request a service increase [here](https://us-east-1.console.aws.amazon.com/servicequotas/home/services)

In [8]:
model_id, model_version = "huggingface-textgeneration1-gpt-j-6b", "*"

from sagemaker import image_uris, model_uris, script_uris, hyperparameters

# you can change the instance to a smaller instance - https://aws.amazon.com/ec2/instance-types/p3/
# training_instance_type = "ml.p3dn.24xlarge" 

training_instance_type = "ml.g4dn.16xlarge"

# G4 instance https://aws.amazon.com/ec2/instance-types/g4/
# training_instance_type = "ml.g4dn.12xlarge" 

# training_instance_type = "ml.g4dn.4xlarge"

# Retrieve the docker image for training
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=model_id,
    model_version=model_version,
    image_scope="training",
    instance_type=training_instance_type,
)

# Retrieve the training script
train_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="training"
)

print(train_source_uri)
# Retrieve the pre-trained model tarball to further fine-tune
train_model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="training"
)

print(train_model_uri)

s3://jumpstart-cache-prod-us-east-1/source-directory-tarballs/huggingface/transfer_learning/textgeneration1/prepack/v1.2.0/sourcedir.tar.gz
s3://jumpstart-cache-prod-us-east-1/huggingface-training/train-huggingface-textgeneration1-gpt-j-6b.tar.gz


## Spot Training configuration
If **use_spot_instances** is set to **True** below training will use spot instances.

Note: If you are using spot instances for training you will need to store training checkpoints in case your spot instances are shutdown. This allows you to continue training where you left off if spot instances are terminated.

In [ ]:
from sagemaker.utils import name_from_base
training_job_name = name_from_base(f"{model_id}-transfer-learning")

# set use_spot_instances to true if you are going to use spot instances for training
# ensure you have the proper quota for the instance type you set for the training_instance_type variable
# you can check your quota here https://us-east-1.console.aws.amazon.com/servicequotas/home/services/sagemaker/quotas 
# by enter the instance type you plan on using
use_spot_instances = True
max_run = 36000 # in seconds
max_wait = 72000 if use_spot_instances else None # in seconds

checkpoint_s3_uri = None

if use_spot_instances:
    # sets the location where training checkpoint will be stored if using spot instances
    checkpoint_s3_uri = f'{s3_training_output_path}/checkpoints/{training_job_name}'
    
print (f'Checkpoint storage location: {checkpoint_s3_uri}')

## Train with Automatic Model Tuning (HPO)
This section configures Automatic Model Tuning if you change from **use_auto_tuning = False** to **use_auto_tuning = True**. By default we set it to false for this example.

In [10]:
from sagemaker import hyperparameters

# Set default hyper-parameters for fine-tuning the model
hyperparameters = hyperparameters.retrieve_default(model_id=model_id, model_version=model_version)

# Overriding default hyperparameters with custom values
hyperparameters["epoch"] = "1"
hyperparameters["per_device_train_batch_size"] = "4"

# If you are training with domain specific datasets you will need parameter to set this to False
hyperparameters["instruction_tuned"] = False

print(hyperparameters)

{'epoch': '1', 'learning_rate': '6e-06', 'per_device_train_batch_size': '4', 'per_device_eval_batch_size': '8', 'warmup_ratio': '0.1', 'instruction_tuned': False, 'train_from_scratch': 'False', 'fp16': 'True', 'bf16': 'False', 'evaluation_strategy': 'steps', 'eval_steps': '20', 'gradient_accumulation_steps': '2', 'logging_steps': '10', 'weight_decay': '0.2', 'load_best_model_at_end': 'True', 'max_train_samples': '-1', 'max_val_samples': '-1', 'seed': '10', 'max_input_length': '-1', 'validation_split_ratio': '0.2', 'train_data_split_seed': '0', 'preprocessing_num_workers': 'None', 'max_steps': '-1', 'gradient_checkpointing': 'True', 'early_stopping_patience': '3', 'early_stopping_threshold': '0.0', 'adam_beta1': '0.9', 'adam_beta2': '0.999', 'adam_epsilon': '1e-08', 'max_grad_norm': '1.0', 'label_smoothing_factor': '0', 'logging_first_step': 'False', 'logging_nan_inf_filter': 'True', 'save_strategy': 'steps', 'save_steps': '500', 'save_total_limit': '1', 'dataloader_drop_last': 'False',

## Set hyperparameters
This section configures any hyperparameter if you decide to use automated model tuning. In this example we are not but the code exist if you would like to test out automated model tuning.

In [11]:
from sagemaker.tuner import ContinuousParameter

# Use AMT (automated model tuning) for tuning and selecting the best model
use_auto_tuning = False

# Define objective metric, based on which the best model will be selected.
amt_metric_definitions = {
    "metrics": [{"Name": "eval:loss", "Regex": "'eval_loss': ([0-9]+\.[0-9]+)"}],
    "type": "Minimize",
}

# You can select from the hyperparameters supported by the model, and configure ranges of values to be searched for training the optimal model.(https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-define-ranges.html)
hyperparameter_ranges = {
    "learning_rate": ContinuousParameter(0.00001, 0.0001, scaling_type="Logarithmic")
}

# Increase the total number of training jobs run by AMT, for increased accuracy (and training time).
max_jobs = 2
# Change parallel training jobs run by AMT to reduce total training time, constrained by your account limits.
# if max_jobs=max_parallel_jobs then Bayesian search turns to Random.
max_parallel_jobs = 2

## Start Training
Here we start our SageMaker training job to tune the model. Depending on how much data is being used, the size of your training instance and the number of instances used for training will dictate how long it will take to train/tune your new model.

If your training job fails because you surpassed your qouta for that instance type you can request an increase in your quota for that instance type [here](https://us-east-1.console.aws.amazon.com/servicequotas/home/services/sagemaker/quotas). You can request an instance quota increase for regular training instances and spot instances.

You may also run into an error stating lack of capacity for your instance type. If you receive this type of error you can re-run the cell until the training job starts.



In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.tuner import HyperparameterTuner

# the default value is File which downloads the entire dataset to the instance before training starts
# setting input_mode to Pipe allows you to stream the training data to the instance during training
# training_file_input_mode = "Pipe"

# defines model metrics that are used to evaluate the models performance
metric_definitions = [
    {"Name": "train:loss", "Regex": "'loss': ([0-9]+\.[0-9]+)"},
    {"Name": "eval:loss", "Regex": "'eval_loss': ([0-9]+\.[0-9]+)"},
    {"Name": "eval:runtime", "Regex": "'eval_runtime': ([0-9]+\.[0-9]+)"},
    {"Name": "eval:samples_per_second", "Regex": "'eval_samples_per_second': ([0-9]+\.[0-9]+)"},
    {"Name": "eval:eval_steps_per_second", "Regex": "'eval_steps_per_second': ([0-9]+\.[0-9]+)"},
]

# Create SageMaker Estimator instance
tg_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    volume_size=50,
    instance_type=training_instance_type,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    base_job_name=training_job_name,
    metric_definitions=metric_definitions,
    checkpoint_s3_uri=checkpoint_s3_uri,
    use_spot_instances=use_spot_instances,
    max_run=max_run,
    max_wait=max_wait
)

# checks to see if you are using automated model tuning
if use_auto_tuning:
    hp_tuner = HyperparameterTuner(
        tg_estimator,
        amt_metric_definitions["metrics"][0]["Name"],
        hyperparameter_ranges,
        amt_metric_definitions["metrics"],
        max_jobs=max_jobs,
        max_parallel_jobs=max_parallel_jobs,
        objective_type=amt_metric_definitions["type"],
        base_tuning_job_name=training_job_name
    )
    
    print("Using hyerparameter tuning job")
    # Start a SageMaker Tuning job to search for the best hyperparameters
    hp_tuner.fit({"train": s3_training_location, "validation": s3_validation_location }, logs=True)
else:
    print(f"Training file location is {s3_training_location}")
    print(f"Validation file location is {s3_validation_location}")
    
    # Start a SageMaker Training job by passing s3 path for the training dataset
    tg_estimator.fit(
        {"train": s3_training_location, "validation": s3_validation_location}, logs=True
    )

INFO:sagemaker:Creating training-job with name: huggingface-textgeneration1-gpt-j-6b-tr-2023-08-07-03-51-49-027


2023-08-07 03:51:49 Starting - Starting the training job...
2023-08-07 03:52:03 Starting - Preparing the instances for training......
2023-08-07 03:53:09 Downloading - Downloading input data.........................................................
2023-08-07 04:02:43 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-08-07 04:02:44,765 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-08-07 04:02:44,783 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-08-07 04:02:44,793 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-08-07 04:02:44,795 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-08-07 04:02:47,368 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt

## Review Training metrics
Here we output the training metrics

In [ ]:
from sagemaker import TrainingJobAnalytics

if use_auto_tuning:
    print("get the best trained model from the hyperparameter tuner")
    training_job_name = hp_tuner.best_training_job()
else:
    print("using estimator model")
    training_job_name = tg_estimator.latest_training_job.job_name

df = TrainingJobAnalytics(training_job_name=training_job_name).dataframe()
df.head(10)

## Deploy & run Inference on the fine-tuned model
In this section we are deploying a model inference endpoint so that we can run inferences against the fine-tuned model.

In [ ]:
inference_instance_type = "ml.g4dn.12xlarge"

# Retrieve the docker container uri for inference
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)

endpoint_name_after_finetune = name_from_base(f"fine-tuned-{model_id}")

print(f"Endpoint name: {endpoint_name_after_finetune}" )

# Deploy to SageMaker inference endpoint
finetuned_predictor = (hp_tuner if use_auto_tuning else tg_estimator).deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    image_uri=deploy_image_uri,
    endpoint_name=endpoint_name_after_finetune
)

## Inference Helper functions
Creates two helper functions that will be used when we call the inference endpoint

In [ ]:
import json
import boto3

def query_endpoint_with_payload(encoded_json, endpoint_name):
    client = boto3.client("runtime.sagemaker")
    
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/json", Body=encoded_json
    )
    
    return response


def parse_response_texts(query_response):
    generated_text = []
    model_predictions = json.loads(query_response["Body"].read())
    return model_predictions[0]

In [ ]:
parameters = {
    "max_length": 500,
    "num_return_sequences": 1,
    "top_k": 250,
    "top_p": 0.8,
    "do_sample": True,
    "temperature": 1,
}

res_gpt_finetune = []
    
for quota_text in [
    "Tell me about the Matrimonial and Family Proceedings Act 1984",
    "tell me about the Ellenborough Park [1956] case",
    # "what was the case about that involved The Palmers Wood Oil Field",
    # "what was the case about that involved Mohammed Jabar Ahmed, Mohammed Azmir Khan and Michael Marteen",
    # "Tel me about Thirteenth Protocol to the European Convention on Human Rights (2004)",
]:
    
    payload = {"text_inputs": f"{quota_text}:", **parameters}
    
    query_response = query_endpoint_with_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name_after_finetune
    )
    
    generated_texts = parse_response_texts(query_response)[0]["generated_text"]
    res_gpt_finetune.append(generated_texts)
    
    print(generated_texts)
    print("\n------------------------------------------------------------------")

## Clean-Up
Here we are performing clean-up by deleting the fine-tuned model and deleting the inference endpoint that was deployed.

Note: Leaving an inference endpoint running can be costly depending on the instance type you deployed your endpoint to. In this notebook we are using the ml.g5.12xlarge instance type for our inference endpoint. This is a GPU based instance which cost roughly $5.672 per hour to run at this time of publishing this notebook.

In [ ]:
# Delete the SageMaker endpoint and the attached resources
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()